In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Настройки отображения
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

## Сбор всех детализированных отчетов WB в один датасет

In [ ]:
folder_path = "Путь к папке с отчетами"

def combine_reports(folder_path):
    df_combined = pd.DataFrame()
    for file in os.listdir(folder_path):
        if file.endswith('.xlsx'):
            try:
                df_temp = pd.read_excel(os.path.join(folder_path, file))

                # Чистка пустых и не нужных столбцов, замена пустых значений на 'empty'
                df_temp = df_temp.drop(df_temp.columns[[0, 2, 3, 4, 7, 8, 16, 17, 18, 20, 21, 39, 43, 46, 47, 48, 51, 52, 54, 55, 56]], axis=1)
                df_temp = df_temp.replace(np.nan, 'empty')

                df_combined = pd.concat([df_combined, df_temp], ignore_index=True)
            except Exception as e:
                print(f"Ошибка при обработке файла {file}: {e}")
    return df_combined

data = combine_reports(folder_path)

#Форматирование данных для добавления регионов ПВЗ
data = data.rename(columns={'Номер офиса':'id', 'Тип документа':'type_doc', 'Обоснование для оплаты':'found_doc','Артикул поставщика':'Артикул'})
data['Артикул'] = data['Артикул'].str.upper()

#Замена id ПВЗ на их регионы
data_regions = pd.read_excel("База ПВЗ.xlsx")
data_regions = data_regions.drop('Unnamed: 0', axis=1)
data = pd.merge(data, data_regions[['id', 'region']], on='id', how='left')
data['id'] = data['region'].fillna(data['id'])
data = data.drop('region', axis=1)


#Отбираем только актуальные столбцы для анализа
data_dashboard = data[['Артикул', 'Название', 'type_doc', 'found_doc', 'Дата заказа покупателем', 'Дата продажи', 'Кол-во', 'Цена розничная', 'Вайлдберриз реализовал Товар (Пр)', 'К перечислению Продавцу за реализованный Товар', 
                       'Услуги по доставке товара покупателю', 'Общая сумма штрафов', 'id', 'Склад', 'Страна', 'Возмещение издержек по перевозке/по складским операциям с товаром', 'Хранение', 'Удержания', 'Платная приемка']]

data_dashboard['id'] = data_dashboard['id'].apply(lambda x: "Без региона" if isinstance(x, (int, float)) or x == 'empty' else x)
data_dashboard['id'] = data_dashboard['id'].replace({'Абай':'Казахстан', 'Акмолинская':'Казахстан', 'Актюбинская':'Казахстан',
                                        'Алматинская':'Казахстан', 'Алматы':'Казахстан', 'Араратская':'Армения', 
                                        'Арагацотнская':'Армения', 'Армавирская':'Армения', 'Астана':'Казахстан', 
                                        'Атырауская':'Казахстан', 'Байконур':'Казахстан', 'Бишкек':'Кыргызстан',
                                        'Брестская':'Беларусь', 'Вайоцдзорская':'Армения', 'Витебская':'Беларусь',
                                        'Восточно-Казахстанская':'Казахстан', 'Гехаркуникская':'Армения',
                                        'Гомельская':'Беларусь', 'Гродненская':'Беларусь', 'Ереван':'Армения', 
                                        'Жамбылская':'Казахстан', 'Жетысу':'Казахстан', 'Западно-Казахстанская':'Казахстан',
                                        'Иссык-Кульская область':'Кыргызстан', 'Карагандинская':'Казахстан',
                                        'Костанайская':'Казахстан', 'Котайкская':'Армения', 'Кызылординская':'Казахстан',
                                        'Лорийская':'Армения', 'Мангистауская':'Казахстан', 'Минск':'Беларусь',
                                        'Минская':'Беларусь', 'Могилёвская ':'Беларусь', 'Ош':'Кыргызстан',
                                        'Павлодарская':'Казахстан', 'Северо-Казахстанская область':'Казахстан', 
                                        'Сюникская':'Армения', 'Тавушская':'Армения', 'Ташкент':'Узбекистан',                         
                                        'Туркестанская область':'Казахстан', 'Улытау':'Казахстан',
                                        'Восточно-Казахстанская':'Казахстан', 'Чуйская':'Кыргызстан',
                                        'Ширакская':'Армения', 'Шымкент':'Казахстан', 'Эвенкийский':'Красноярский', 
                                        'Южно-Казахстанская область':'Казахстан'})

data_dashboard = data_dashboard.rename(columns={'type_doc':'Тип документа', 'found_doc':'Обоснование для оплаты', 'id':'Регион ПВЗ'})

#Необходимо присоединить группу и категорию товаров
nomeklatura = pd.read_excel("АССОРТИМЕНТНЫЙ ПЕРЕЧЕНЬ.xlsx")
data_dashboard = pd.merge(data_dashboard, nomeklatura, on='Артикул', how='left')
data_dashboard = data_dashboard[['Артикул', 'Название', 'Категория', 'Группа товара', 'Подгруппа товара', 'Тип документа', 'Обоснование для оплаты', 
                                 'Дата заказа покупателем', 'Дата продажи', 'Кол-во', 'Цена розничная', 'Вайлдберриз реализовал Товар (Пр)', 'К перечислению Продавцу за реализованный Товар', 
                                 'Услуги по доставке товара покупателю', 'Общая сумма штрафов', 'Регион ПВЗ', 'Склад', 'Страна',
                                 'Возмещение издержек по перевозке/по складским операциям с товаром', 'Хранение', 'Удержания', 'Платная приемка']]

#Смена форматов дат
data_dashboard['Дата заказа покупателем'] = pd.to_datetime(data_dashboard['Дата заказа покупателем'], format='%Y-%m-%d', errors='coerce')
data_dashboard['Дата продажи'] = pd.to_datetime(data_dashboard['Дата продажи'], format='%Y-%m-%d', errors='coerce')

# Сохранение данных для последующих действий
data_dashboard.to_csv('data_WB.csv')

## Пример дальнейших операций: формирование отчета с данными о продажах

In [ ]:
data_income = data.query('found_doc == "Продажа"')
data_income = pd.merge(data_income, nomeklatura, on='Артикул', how='left')
data_income = data_income[['Артикул', 'Название', 'Категория', 'Группа товара', 'Подгруппа товара', 'Дата заказа покупателем', 
                           'Дата продажи','Кол-во', 'Цена розничная', 'Вайлдберриз реализовал Товар (Пр)', 'id', 'Склад', 'Страна']]
data_income['Дата продажи'] = pd.to_datetime(data_income['Дата продажи'], format='%Y-%m-%d')
data_income['Дата заказа покупателем'] = pd.to_datetime(data_income['Дата заказа покупателем'], format='%Y-%m-%d')

data_income.to_csv("Sales_WB.csv")